In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
# Install pyspark
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 18.6 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824025 sha256=4bc5f42c64961e5fefbed60f6acafe8372e832ecd2f4e160e2014e2d84e589ec
  Stored in directory: /root/.cache/pip/wheels/6c/e3/9b/0525ce8a69478916513509d43693511463c6468db0de237c86
Successfully built pyspark


In [3]:
# task 1_1.py

from pyspark import SparkConf, SparkContext
import requests
conf = SparkConf().setAppName("GroceriesData")
sc = SparkContext.getOrCreate(conf)

def load_groceries_data(link_csv = 'https://raw.githubusercontent.com/stedy/Machine-Learning-with-R-datasets/master/groceries.csv'):

  """  
  This function loads the csv file to spark rdd api.
  Args: 
    link_csv: input file path mapped from github 
  Returns:
    This function returns spark rdd api variable
  """

  groceries_data_rdd = sc.parallelize(requests.get(link_csv).text.split("\n"))
  return groceries_data_rdd

load_groceries_data()


ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:274

In [4]:
# task 1_2.py
# Tash 2
# =========
from pyspark import SparkConf, SparkContext
import requests
conf = SparkConf().setAppName("GroceriesData")
sc = SparkContext.getOrCreate(conf)

def write_out_1_2ab(write_path_1_2a = 'out/out_1_2a.txt', write_path_1_2b ='out/out_1_2b.txt'):

  """ 
  This function writes the output by executing functions unique_products and total_count.
  Args: 
    write_path_1_2a: output file path for Part 1 task 2 A
    write_path_1_2b: output file path for Part 1 task 2 B
  Returns:
    None
  """

  unique_products_val = unique_products()

  total_count_val = total_count()

  unique_products_val.saveAsTextFile(write_path_1_2a)

  sc.parallelize(["Count: {}".format(total_count_val)]).saveAsTextFile(write_path_1_2b)


def unique_products():

  """  
  unique_products
  This function takes the input by excuting load_groceries_data and returns the unique products from the list.
  Args: 
    None 
  Returns:
    This functions returns unique products from the list 
  """

  groceries_data_rdd = load_groceries_data()

  unique_products_val = groceries_data_rdd.flatMap(lambda x: x.split(',')).distinct()

  return unique_products_val

 
def total_count():

  """  
  This function takes the input by excuting load_groceries_data and returns total count of the list.
  Args: 
    None 
  Returns:
    This function returns total count of values from the list
  """

  groceries_data_rdd = load_groceries_data()

  total_count_val = groceries_data_rdd.flatMap(lambda x: x.split(',')).count()

  return total_count_val



write_out_1_2ab()

In [5]:
# task 1_3.py
# Task 3
#==========
from pyspark import SparkConf, SparkContext
import requests
conf = SparkConf().setAppName("GroceriesData")
sc = SparkContext.getOrCreate(conf)

def write_out_1_3(write_path_1_3 = 'out/out_1_3.txt'):

  """  
  This function writes the output by executing functions item_counts.
  Args: 
    write_path_1_3: output file path for Part 1 task 3
  Returns:
    None
  """

  item_counts_val = item_counts()
  
  sc.parallelize(item_counts_val).saveAsTextFile(write_path_1_3)

def item_counts():

  """  
  This function loads the csv file to spark rdd api and takes the top 5 products based on the cout of their occurance in the list.
  Args: 
    None
  Returns:
    This function returns top 5 products in a tuple.
  """
  
  groceries_data_rdd = load_groceries_data()

  item_counts_val = groceries_data_rdd.flatMap(lambda x: x.split(',')) \
                        .map(lambda x: (x.strip(), 1)) \
                        .reduceByKey(lambda a, b: a + b) \
                        .sortBy(lambda x: -x[1]) \
                        .take(5)
  return item_counts_val

write_out_1_3()